# @todo
=== MAKE THE CONVERSION OF THE DATABASE ELEMENTS TO A DICTIONARY, MAINLY FOR THE MENU TABLE

--- MAKE A SAMPLE RECOMMENDER USING THE PREVIOUS DATASET COLLECTED USING GOOGLE FORMS ON THE LINUX COMPUTER

--- ADD NEW USERS TO DB AS SHOWN IN THE DATASET AND THEN IMPLEMENT THE RECOMMENDER SYSTEM THROUGH THE DATABASE ELEMENTS

--- FINAL BUG CHECK AND NEW USER TEST

---DEPLOYMENT

In [1]:
from chuna_lagake import models, routes, forms, app, db
from chuna_lagake.models import User, Menu, Ratings

In [2]:
mahesh = User.query.filter_by(username='mahesh').first()

In [78]:
menu = []
for i in range(50):
    item = Menu.query.filter_by(id='{}'.format(i+1)).first()
    menu.append([item.id, item.name, item.features])
# menu

In [32]:
# USER DATA
user = []
for i in range(User.query.count()):
    user_dict = {}
    User_Data = User.query.filter_by(id='{}'.format(i+1)).first()
    user_dict.update({'User_ID':User_Data.id, 'Name':User_Data.username})
    user.append(user_dict)
user

[{'User_ID': 1, 'Name': 'mahesh'},
 {'User_ID': 2, 'Name': 'User2'},
 {'User_ID': 3, 'Name': 'user3'},
 {'User_ID': 4, 'Name': 'user4'},
 {'User_ID': 5, 'Name': 'user5'}]

In [24]:
# RATINGS
rating = []
for i in range(Ratings.query.count()):
    rating_dict = {}
    Rating = Ratings.query.filter_by(id='{}'.format(i+1)).first()
    rating_dict.update({'User_ID':Rating.user_id, 'Item_ID':Rating.item_id, 'Rating':Rating.rating})
    rating.append(rating_dict)
rating

[{'User_ID': 1, 'Item_ID': 2, 'Rating': 4.0},
 {'User_ID': 1, 'Item_ID': 12, 'Rating': 3.0}]

In [6]:
item_features = ['Banaras','Calcutta','Maghai','Sada','Meetha','Chocolate','Dry Fruit','Mango','Strawberry','Pineapple','Gold',
 'Kaju','Jelly','Rose','Shahi','Kesar','Vanilla','Masala','Khatta','Orange','White','Silver','RaatRani','Nutella','Special']

In [21]:
# ITEM_FEATURES
items = []
for item in menu:
    item_dict = {}
    item_dict.update({'Name':item[1], 'Item_ID':item[0]})
    for i in range(25):
        item_dict.update({item_features[i]:item[2][i]})
    items.append(item_dict)
print(items[0])

{'Name': 'Banaras Sada Paan', 'Item_ID': 1, 'Banaras': 1.0, 'Calcutta': 0.0, 'Maghai': 0.0, 'Sada': 1.0, 'Meetha': 0.0, 'Chocolate': 0.0, 'Dry Fruit': 0.0, 'Mango': 0.0, 'Strawberry': 0.0, 'Pineapple': 0.0, 'Gold': 0.0, 'Kaju': 0.0, 'Jelly': 0.0, 'Rose': 0.0, 'Shahi': 0.0, 'Kesar': 0.0, 'Vanilla': 0.0, 'Masala': 1.0, 'Khatta': 1.0, 'Orange': 0.0, 'White': 0.0, 'Silver': 0.0, 'RaatRani': 0.0, 'Nutella': 0.0, 'Special': 0.0}


In [49]:
# Create the Model
from lightfm import LightFM
from lightfm.data import Dataset

dataset = Dataset()

dataset.fit((x['User_ID'] for x in rating),
           (x['Item_ID'] for x in rating))

In [50]:
num_users, num_items = dataset.interactions_shape()
print("Num Users: {}, Num Items: {}".format(num_users, num_items))

Num Users: 1, Num Items: 2


In [51]:
dataset.fit_partial(items=(x['Item_ID'] for x in items),
                   users=(x['User_ID'] for x in user))    

In [52]:
def add_item_features(feature):
    dataset.fit_partial(items=(x['Item_ID'] for x in items),
                       item_features=(x[feature] for x in items))

In [53]:
for feature in item_features:
    add_item_features(feature)

In [54]:
# Build interactions

In [55]:
(interactions, weights) = dataset.build_interactions(((x['User_ID'], x['Item_ID']) for x in rating))
print(repr(interactions))

<5x50 sparse matrix of type '<class 'numpy.int32'>'
	with 2 stored elements in COOrdinate format>


In [79]:
item_features = dataset.build_item_features(((x['Item_ID'],[x['Banaras'], x['Calcutta'], x['Maghai'], x['Sada'],
                                              x['Meetha'], x['Chocolate'], x['Dry Fruit'], x['Mango'], x['Strawberry'],
                                              x['Pineapple'], x['Gold'], x['Kaju'], x['Jelly'], x['Rose'], x['Shahi'],
                                              x['Kesar'], x['Vanilla'], x['Masala'], x['Khatta'], x['Orange'], x['White'],
                                              x['Silver'], x['Nutella'], x['RaatRani'], x['Special']]) for x in items))

In [61]:
model = LightFM(loss='warp')
model.fit(interactions, item_features=item_features)

In [64]:
import numpy as np
labels = np.array([x['Name'] for x in items])
labels.shape

(50,)

In [75]:
positives = []
recommended = []
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data.shape
    
    for user_id in user_ids:
        known_positives = labels[data.tocsr()[user_id-1].indices]
        scores = model.predict(user_id-1, np.arange(n_items))
        top_items = labels[np.argsort(-scores)]
        
        print("User IDS: {}".format(user_id))
        print("------Known Positives:")
        
        for x in known_positives[:3]:
            positives.append(x)
            print("--------{}".format(x))
            
        print("\n------Recommended:")
        for x in top_items[:3]:
            recommended.append(x)
            print("--------{}".format(x))
            
        

In [76]:
sample_recommendation(model, interactions, [1, 2])

User IDS: 1
------Known Positives:
--------Banaras Sada Paan
--------Calcutta Sada Paan

------Recommended:
--------Banaras Meetha Paan
--------Calcutta Sada Paan
--------Banaras Sada Paan
User IDS: 2
------Known Positives:

------Recommended:
--------Banaras Meetha Paan
--------Calcutta Sada Paan
--------Calcutta Nutella Paan
